In [1]:
"""
This code is for generating an amalgamated dataset of fields of potential vorticity 
cutoffs from the dataset published in Portmann et al. 2021 (https://doi.org/10.5194/wcd-2-507-2021). 

It opens the netcdf files of pv cutoff areas from Irina Rudeva's directory on Gadi, extracts the
fields from the specified range of isentropic levels, and saves the output also as a netcdf file. 
"""
import numpy as np
import xarray as xr

In [2]:
def monthstr(m):
    '''This function converts a number representing a month to a string, and adds a leading
    0 if the month number is between 1 and 9. '''
    
    if m < 10:
        mstr = str(0) + str(m)
    else:
        mstr = str(m)
    return mstr

In [3]:
# ============================== Setup Parameters ===========================

years = (1979, 2017)
months = (1,12)

levels = (300, 320)

portmann_data_path = '/g/data/eg3/ir9737/cutoff/data/all/'
save_path = '/g/data/w40/nxg561/Portmann_Cutoffs/'

In [4]:
%%time

# Loop through each month of each year, open all the pv cutoff files, and stack the
# cutoff fields into a single file for that month.

yearrange = np.arange(years[0], years[1] + 1)
monthrange = np.arange(months[0], months[1] + 1)

for year in yearrange:
    print(year)
    for month in monthrange:

        #print(year, ' ', month)
        
        ### Extract the Portmann cutoff low fields for each timestep of the month ###
        # Open the Portman low file one by one
        portmann_file_path = portmann_data_path + str(year) + '/' + monthstr(month) + '/'
        portmann_file_name = 'M' + str(year) + monthstr(month) + '*'
        xr_obj = xr.open_mfdataset(portmann_file_path + portmann_file_name, combine = 'nested', concat_dim = 'time', engine = 'netcdf4')

        # This was adapted from Irina's code to correct the axes. It creates and assigns new lat, lon and level axes
        xr_obj_upd = xr_obj.squeeze().rename_dims({"dimz_C":"level","dimy_C":"latitude","dimx_C":"longitude"},).assign_coords(
                             longitude = np.arange(-180,181,1)
                            ,latitude  = np.arange(-90,91,1)
                            ,level     = np.arange(290,351,5))

        # Select all levels within the specified range, for the southern Hemisphere
        track_id_fields = xr_obj_upd['track_id'].sel(level = slice(levels[0], levels[1]), latitude = slice(-90, 0))

        # Compute a composite sum field for the cutoffs through all vertical levels
        cutoff_fields = track_id_fields.sum(dim = 'level')

        # Save the stack of composite fields in a file for that month.
        output_filename = 'pv_cutoffs_' + str(year) + monthstr(month) + '.nc'
        cutoff_fields.to_netcdf(save_path + output_filename, mode = 'w', format = 'NETCDF4', engine = 'netcdf4')

2009
0.3.0
2010
2011
2012
2013
2014
2015
2016
2017
CPU times: user 2min 53s, sys: 2min 6s, total: 5min
Wall time: 18min 41s


In [5]:
# Finally concatenate all the months into one file for the whole period
xr_obj_final = xr.open_mfdataset(save_path + '*.nc', combine = 'nested', concat_dim = 'time', engine = 'netcdf4')

output_filename = 'pv_cutoffs_1979-2017_old.nc'
        
xr_obj_final.to_netcdf(save_path + output_filename, mode = 'w', format = 'NETCDF4', engine = 'netcdf4')

In [ ]:
cutoff_fields.data

In [ ]:
xr_obj_final